In [1]:
# Eran Mukamel, February 2021
# 
# Add a gene-by-cell matrix (gmat) to a .snap file. This should be equivalent to:  
#   snaptools snap-add-gmat
#
# Usage: snap2gmat <snap-file> <genes-file>
#
# Inputs:
#  snap_file = path to .snap file
#  genes_file = path to bed file of genes 
#
# Output
#  Saves a file <snap-file>.gmat.npz

import pandas as pd
import numpy as np
import re,os,sys
import h5py,time
from scipy.sparse import coo_matrix

In [13]:
# snap_file = sys.argv[1]
# genes_file = sys.argv[2]

snap_file = "CEMBA190214_11E.snap"
genes_file = "Annotation/gencode.vM16.annotation_up10kb.bed"
# genes_file = "Annotation/gencode.vM16.annotation_pc_genes.tsv"

print('*** Creating gene-by-cell matrix (gmat) from snap file')
print('*** Processing %s:' % snap_file)

# TODO: Filter out cells with < 1000 reads

# My attempt at a faster implementation of snaptools snap-add-gmat
def assign_genes(filename,
                 genes,
                 istart=None,
                 istop=None,
                 verbose=False):
    from scipy.sparse import coo_matrix
    t0=time.time()
    with h5py.File(filename,'r') as file:
        if istart is None:
            istart=0
        if istop is None:
            istop=file['FM/fragStart'].shape[0]

        # Assign every fragment to a gene. NOTE: This will assign every fragment to ONE gene, even if it overlaps multiple genes
        fragChrom = file['FM/fragChrom'][istart:istop].astype(str)
        fragStart = file['FM/fragStart'][istart:istop]
        fragEnd = fragStart + file['FM/fragLen'][istart:istop]

        # Figure out the barcode index for each fragment
        bc_pos = file['FM/barcodePos'][:]-1
        bc_pos = np.append(bc_pos,np.inf)
        fragBC = np.digitize(istart+np.arange(len(fragChrom)), bc_pos)
        nbc = file['FM/fragChrom'].shape[0]
        # TEMPORARY: Create smaller dataset for testing    
        nbc = min(nbc, 100000)
    if verbose:
        print('Loaded data; t=%3.3f s' % (time.time()-t0))
    
    t0=time.time()
    fragGene = np.empty(len(fragEnd))
    i,j = np.array([],dtype=int),np.array([],dtype=int)
    chroms = genes['chr'].unique()
    for chrom in chroms:
        gu = genes[genes['chr']==chrom]
        fu = fragChrom==chrom
        if fu.any():
            fstart = fragStart[fu]
            fend = fragEnd[fu]
            fgene = fragGene[fu]
            fbc = fragBC[fu]
            
            for gi in range(gu.shape[0]):
                overlap_barcodes = fbc[(fend>gu.iloc[gi]['start']) & (fstart<gu.iloc[gi]['end'])]
                if len(overlap_barcodes)>0:
                    i=np.append(i,[gu.index[gi]]*len(overlap_barcodes))
                    j=np.append(j,overlap_barcodes)
        if verbose:
            print('%s, %d genes, %d fragments, %d total overlaps, t=%3.3fs' % (chrom,gu.shape[0],fu.sum(),len(i),time.time()-t0))
    ngenes = genes.shape[0]
    gmat = coo_matrix((np.ones(len(i)), (i,j-1)), shape=(ngenes,nbc))
    return gmat

# 15 s for 10,000 fragments
genes = pd.read_csv(genes_file,
                    sep='\t',
                   header=None,
                   names=['chr','start','end','ensid','gene','strand','type'])

with h5py.File(snap_file,'r') as file:
    nbc = file['FM/fragChrom'].shape[0]
    # TEMPORARY: Create smaller dataset for testing    
    nbc = min(nbc, 100000)

chunksize = int(1e6)
t0=time.time()

print(t0, nbc, chunksize)
ngenes = genes.shape[0]
gmat = coo_matrix((ngenes,nbc), np.int)
for istart in np.arange(0,nbc,chunksize):
    gmat += assign_genes(snap_file,genes,istart=istart,istop=istart+chunksize)
    tcurr = time.time()-t0
    print('%d/%d: t=%3.3f s; estimated total time = %3.3f min' % (istart+chunksize,nbc,tcurr,
                                                           tcurr*nbc/(istart+chunksize)/60))
    
    
from scipy.sparse import save_npz
save_npz(snap_file.replace('.snap','.gmat.npz'), gmat)
print('Saved output file %s ' % snap_file.replace('.snap','.gmat.npz'))

*** Creating gene-by-cell matrix (gmat) from snap file
*** Processing CEMBA190214_11E.snap:
1617139085.7276356 100000 1000000
1000000/100000: t=233.878 s; estimated total time = 0.390 min
Saved output file CEMBA190214_11E.gmat.npz 


In [ ]:
# TODO: Select HVF from genes

In [14]:
# Aditya: Save anndata
import anndata as ad

with h5py.File(snap_file,'r') as file:
    print(file['BD']['name'])

# TODO: Set obs to numerical index
n_genes, n_obs = gmat.shape

obs_df = pd.DataFrame(index=pd.RangeIndex(start=0, stop=n_obs))

print(gmat.T.shape, len(genes))
adata = ad.AnnData(X = gmat.T,
                   obs = obs_df,
                   var = genes,
                   )

results_file = snap_file.replace('.snap', '.gmat.h5ad')
adata.write_h5ad(results_file)

Transforming to str index.


<HDF5 dataset "name": shape (13257,), type "|O">
(100000, 53379) 53379


Transforming to str index.
... storing 'chr' as categorical
... storing 'gene' as categorical
... storing 'strand' as categorical
... storing 'type' as categorical


<HDF5 dataset "name": shape (13257,), type "|O">
